In [1]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [2]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
train = pd.read_json("input/train.json")
#test = pd.read_json("input/test.json")

In [12]:
only_band1 = ['band1_'+str(i) for i in range(75*75)]
only_band2 = ['band2_'+str(i) for i in range(75*75)]
both_bands = only_band1+only_band2

In [ ]:
# df = pd.concat([pd.DataFrame(test['band_1'].values.tolist(),columns = only_band1),
#                 pd.DataFrame(test['band_2'].values.tolist(),columns = only_band2)],axis = 1)

In [13]:
def collect_test():
    test_data = pd.DataFrame()
    for i in range(4):
        test_piece = pd.read_csv('input/test_subparts/test'+str(i+1)+'.csv')
        test_data = pd.concat([test_data,test_piece])
        test_data = test_data.reset_index().drop(['index'],axis = 1)
    return test_data

In [14]:
test_data = collect_test()

In [15]:
X_band_test_1=np.array(test_data[only_band1]).reshape(8424,75,75)
X_band_test_2=np.array(test_data[only_band2]).reshape(8424,75,75)
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [4]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [ ]:
# X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
# X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
# X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [ ]:

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=2,stratify = target_train, train_size=0.80)

### Cross Validation Folds

In [5]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches


In [6]:

def cnnmodel():
    X_input = Input(shape = (75,75,3))
    #First composite layer
    X = Conv2D(128,kernel_size = (5,5))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Second composite layer
    X = Conv2D(256,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Third composite layer
    X = Conv2D(256,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Forth composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Flatten layer
    X = Flatten()(X)
    
    #First dense layer
    X = Dense(512,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Second dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Decision layer
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=X_input,outputs=X)

    return model

def model_compile(model,lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None):
   
    if weights_path:
        model.load_weights(weights_path)
    else:
        pass
    if freezing_layers:
        for layer in model.layers[:freezing_layers]:
            layer.trainable = False
    else:
        pass
    optimizer = Adam(lr = lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7, monitor = 'val_loss', verbose = 1, save_best_only = True):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, monitor=monitor, verbose = verbose, save_best_only=True)
    return [es,msave]


In [7]:

def normal_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit(X_train_cv, y_train_cv,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result

def data_augment_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit_generator(data_augmentation(X_train_cv,y_train_cv,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result


In [ ]:
Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5', patience=100, save_best_only = False)
result = normal_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5')
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2_stage2.hdf5', patience=30, save_best_only = True)
result = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel.load_weights('model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5')
Cnnmodel.evaluate(X_valid, y_valid)
#Cnnmodel.evaluate(X_train_cv, y_train_cv)

In [ ]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [ ]:
save_history(result_Cnn)

### Prediction

In [8]:
def Model7_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        global X_train_cv
        X_train_cv = X_train[train_idx]
        global y_train_cv
        y_train_cv = target_train[train_idx]
        global X_valid
        X_valid = X_train[valid_idx]
        global y_valid
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path_stage1 = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv'+str(j+1)+'_stage1.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
        callbacks = get_callbacks(filepath = file_path_stage1, patience=20, save_best_only = True)
        result1 = normal_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)
        print('Satge 1,done!')
        
        Cnnmodel.load_weights(filepath=file_path_stage1)
        #Getting Training Score
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        
        file_path_stage2 = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv'+str(j+1)+'_stage2.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = file_path_stage1)
        callbacks = get_callbacks(filepath = file_path_stage2, patience=30, save_best_only = True)
        result2 = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)
        print('Satge 2,done!')
#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        Cnnmodel.load_weights(filepath=file_path_stage2)
        #Getting Training Score
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
#         temp_test=Cnnmodel.predict(X_test)
#         y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

#     y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_valid_pred_log

In [9]:
Model7_CV(X_train,K = 5)


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_1 (Activation)    (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_2 (Activation)    (None, 33, 33, 256)

Epoch 22/100
Epoch 00022: val_loss improved from 0.30971 to 0.30447, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5
 - 12s - loss: 0.2900 - acc: 0.8744 - val_loss: 0.3045 - val_acc: 0.8602
Epoch 23/100
Epoch 00023: val_loss improved from 0.30447 to 0.28840, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5
 - 12s - loss: 0.2757 - acc: 0.8744 - val_loss: 0.2884 - val_acc: 0.8571
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 12s - loss: 0.2778 - acc: 0.8775 - val_loss: 0.2929 - val_acc: 0.8478
Epoch 25/100
Epoch 00025: val_loss improved from 0.28840 to 0.27390, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5
 - 12s - loss: 0.2772 - acc: 0.8721 - val_loss: 0.2739 - val_acc: 0.8727
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 12s - loss: 0.2435 - acc: 0.8885 - val_loss: 0.3387 - val_acc: 0.8571
Epoch 27/100
Epoc

Epoch 80/100
Epoch 00080: val_loss did not improve
 - 12s - loss: 0.1112 - acc: 0.9579 - val_loss: 0.2358 - val_acc: 0.9068
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 12s - loss: 0.1014 - acc: 0.9532 - val_loss: 0.2517 - val_acc: 0.9068
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 12s - loss: 0.0887 - acc: 0.9602 - val_loss: 0.3238 - val_acc: 0.8913
Satge 1,done!
1282/1282 [==============================] - 3s 3ms/step
Train loss: 0.107827207274
Train accuracy: 0.960998439938
322/322 [==============================] - 1s 2ms/step
Valid loss: 0.233659988605
Valid accuracy: 0.900621118012
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 71, 71, 128)       9728      
________________________________________________________

Epoch 32/100
Epoch 00032: val_loss did not improve
 - 8s - loss: 0.1709 - acc: 0.9276 - val_loss: 0.2336 - val_acc: 0.8975
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 8s - loss: 0.1661 - acc: 0.9329 - val_loss: 0.2426 - val_acc: 0.8975
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 8s - loss: 0.1610 - acc: 0.9337 - val_loss: 0.2703 - val_acc: 0.9037
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 8s - loss: 0.1668 - acc: 0.9217 - val_loss: 0.2411 - val_acc: 0.8975
Epoch 36/100
Epoch 00036: val_loss improved from 0.22384 to 0.22181, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage2.hdf5
 - 8s - loss: 0.1837 - acc: 0.9164 - val_loss: 0.2218 - val_acc: 0.9068
Epoch 37/100
Epoch 00037: val_loss did not improve
 - 8s - loss: 0.1738 - acc: 0.9217 - val_loss: 0.2570 - val_acc: 0.8944
Epoch 38/100
Epoch 00038: val_loss did not improve
 - 9s - loss: 0.1718 - acc: 0.9329 - val_loss: 0.2362 - val_acc: 0.9068
Epoch 39/100
Epoch

 - 16s - loss: 1.0658 - acc: 0.4887 - val_loss: 0.6744 - val_acc: 0.6417
Epoch 2/100
Epoch 00002: val_loss improved from 0.67445 to 0.60280, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv2_stage1.hdf5
 - 12s - loss: 0.6972 - acc: 0.5627 - val_loss: 0.6028 - val_acc: 0.7040
Epoch 3/100
Epoch 00003: val_loss improved from 0.60280 to 0.53432, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv2_stage1.hdf5
 - 12s - loss: 0.6368 - acc: 0.5916 - val_loss: 0.5343 - val_acc: 0.7134
Epoch 4/100
Epoch 00004: val_loss improved from 0.53432 to 0.51629, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv2_stage1.hdf5
 - 12s - loss: 0.5860 - acc: 0.6243 - val_loss: 0.5163 - val_acc: 0.7290
Epoch 5/100
Epoch 00005: val_loss improved from 0.51629 to 0.49413, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv2_stage1.hdf5
 - 12s - loss: 0.5539 - 

...

Epoch 40/100
Epoch 00040: val_loss did not improve
 - 12s - loss: 0.1972 - acc: 0.9159 - val_loss: 0.2547 - val_acc: 0.8875
Epoch 41/100
Epoch 00041: val_loss did not improve
 - 12s - loss: 0.2361 - acc: 0.8902 - val_loss: 0.2382 - val_acc: 0.9031
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 12s - loss: 0.1901 - acc: 0.9174 - val_loss: 0.2367 - val_acc: 0.9000
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 12s - loss: 0.1896 - acc: 0.9073 - val_loss: 0.2581 - val_acc: 0.8938
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 13s - loss: 0.2213 - acc: 0.9065 - val_loss: 0.2503 - val_acc: 0.8969
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 12s - loss: 0.1956 - acc: 0.9104 - val_loss: 0.2278 - val_acc: 0.9062
Epoch 46/100
Epoch 00046: val_loss improved from 0.22598 to 0.22263, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv5_stage1.hdf5
 - 12s - loss: 0.1750 - acc: 0.9237 - val_loss: 0.2226 - val_acc: 0.9062
Epoch 47/10

Epoch 6/100
Epoch 00006: val_loss did not improve
 - 8s - loss: 0.2622 - acc: 0.8752 - val_loss: 0.2093 - val_acc: 0.9156
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 8s - loss: 0.2428 - acc: 0.8789 - val_loss: 0.2166 - val_acc: 0.9219
Epoch 8/100
Epoch 00008: val_loss did not improve
 - 8s - loss: 0.2478 - acc: 0.8857 - val_loss: 0.2231 - val_acc: 0.9125
Epoch 9/100
Epoch 00009: val_loss did not improve
 - 8s - loss: 0.2494 - acc: 0.8865 - val_loss: 0.2137 - val_acc: 0.9313
Epoch 10/100
Epoch 00010: val_loss improved from 0.20901 to 0.19991, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv5_stage2.hdf5
 - 8s - loss: 0.2382 - acc: 0.8896 - val_loss: 0.1999 - val_acc: 0.9250
Epoch 11/100
Epoch 00011: val_loss did not improve
 - 8s - loss: 0.2332 - acc: 0.8986 - val_loss: 0.2000 - val_acc: 0.9219
Epoch 12/100
Epoch 00012: val_loss did not improve
 - 8s - loss: 0.2366 - acc: 0.8926 - val_loss: 0.2036 - val_acc: 0.9250
Epoch 13/100
Epoch 000

Epoch 67/100
Epoch 00067: val_loss did not improve
 - 10s - loss: 0.1696 - acc: 0.9269 - val_loss: 0.2120 - val_acc: 0.9375
Epoch 68/100
Epoch 00068: val_loss improved from 0.17974 to 0.17780, saving model to model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv5_stage2.hdf5
 - 9s - loss: 0.1396 - acc: 0.9398 - val_loss: 0.1778 - val_acc: 0.9375
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 8s - loss: 0.1586 - acc: 0.9347 - val_loss: 0.2240 - val_acc: 0.9250
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 8s - loss: 0.1400 - acc: 0.9422 - val_loss: 0.2267 - val_acc: 0.9313
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 8s - loss: 0.1315 - acc: 0.9466 - val_loss: 0.2054 - val_acc: 0.9156
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 8s - loss: 0.1416 - acc: 0.9375 - val_loss: 0.1868 - val_acc: 0.9187
Epoch 73/100
Epoch 00073: val_loss did not improve
 - 8s - loss: 0.1267 - acc: 0.9497 - val_loss: 0.2259 - val_acc: 0.9313
Epoch 74/100
Epoc

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\metrics\classification.py:1694: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\metrics\classification.py:1694: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


0       1.727947e-04
1       2.552228e-13
2       8.159806e-01
3       1.042914e-03
4       6.815293e-03
5       1.000000e+00
6       9.520487e-01
7       8.554012e-01
8       1.564887e-10
9       3.454490e-02
10      2.754939e-01
11      2.850707e-05
12      7.490485e-01
13      8.624601e-01
14      1.796289e-03
15      6.187331e-03
16      1.549285e-01
17      1.177945e-03
18      2.186298e-01
19      6.577666e-01
20      4.240951e-03
21      4.759872e-01
22      9.214506e-12
23      9.999595e-01
24      6.468930e-10
25      9.999949e-01
26      8.236363e-02
27      8.104882e-05
28      5.450287e-01
29      2.110690e-08
            ...     
1574    2.629913e-01
1575    1.126090e-11
1576    1.171770e-08
1577    1.568383e-02
1578    3.107315e-02
1579    1.433631e-01
1580    5.422156e-02
1581    5.200566e-02
1582    3.458177e-01
1583    5.559990e-01
1584    1.189936e-04
1585    2.436019e-02
1586    8.424275e-01
1587    2.098655e-02
1588    1.028744e-01
1589    4.300445e-05
1590    1.348

### Submission

In [ ]:
def Evaluation_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_test_pred_log=0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)


#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        #Getting Training Score
        print('cv'+str(j+1))
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        print('\n')

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
preds = Evaluation_CV(X_train,K=5)
sub = test[['id']]
sub['is_iceberg'] = preds
sub.to_csv('5cv.csv',index = False)

### Submission(min max median stacking)

In [10]:
def test_scores(K=5):
    y_test_pred_log=0
    test_df = test_data[['id']]
    for j in range(K):
        print('\n===================FOLD=',j+1)
        file_path = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv'+str(j+1)+'_stage2.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = file_path)

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df

In [16]:
test_df = test_scores()


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_41 (Activation)   (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_61 (Dropout)         (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_42 (Activation)   (None, 33, 33, 256)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()



===================FOLD= 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_45 (Activation)   (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_46 (Activation)   (None, 33, 33, 256)


===================FOLD= 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_57 (Activation)   (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_85 (Dropout)         (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_58 (Activation)   (None, 33, 33, 256)

In [17]:
test_df

,id,cv1,cv2,cv3,cv4,cv5
0,5941774d,2.404173e-02,4.497172e-02,4.428454e-02,7.377190e-03,6.259876e-02
1,4023181e,6.732260e-01,4.416436e-01,3.858911e-01,3.992101e-01,5.970809e-01
2,b20200e4,1.251012e-08,2.578506e-08,8.467348e-07,5.958568e-11,1.083537e-07
3,e7f018bb,9.999917e-01,9.999503e-01,9.999803e-01,9.999954e-01,9.999882e-01
4,4371c8c3,4.079539e-02,2.342615e-02,5.530887e-02,1.549587e-03,4.422425e-04
5,a8d9b1fd,5.834405e-02,2.652179e-01,1.165733e-03,4.709214e-02,1.892748e-01
6,29e7727e,9.156364e-02,3.161474e-01,8.323003e-02,2.027565e-01,1.079694e-01
7,92a51ffb,9.999552e-01,9.999814e-01,9.999800e-01,9.999838e-01,9.999992e-01
8,c769ac97,5.200659e-07,4.716208e-06,2.851628e-05,1.757993e-07,2.635662e-09
9,aee0547d,1.418591e-09,7.033230e-10,1.015716e-09,7.863686e-12,1.078889e-14


In [18]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.9,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.1,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),np.median(prediction[prediction.columns[1:]],axis = 1)))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [19]:
sub = get_sub_mmm(test_df)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [20]:
sub

,id,is_iceberg
0,5941774d,7.377190e-03
1,4023181e,4.416436e-01
2,b20200e4,5.958568e-11
3,e7f018bb,9.999954e-01
4,4371c8c3,4.422425e-04
5,a8d9b1fd,5.834405e-02
6,29e7727e,1.079694e-01
7,92a51ffb,9.999992e-01
8,c769ac97,2.635662e-09
9,aee0547d,1.078889e-14
